In [ ]:
"""
Quick visualization of data along a ICESAT2 Track
To understand more about the data being input to this script, consider visiting https://nsidc.org/sites/default/files/atl10-v005-userguide_1_0.pdf
Code written originally by: Chris Polashenski
Refactored, added command line support, and removed sartopy package: John Baker
"""

import h5py
import pandas as pd
import datetime
import numpy as np
import sys
import haversine as hs


In [ ]:
in_file = "D:\\ICEEYE\\Proposal_ID_PP0091278\\CS-13801\\SLEA_3279211_180312\\Baker\\Coincident Data\\ATL10QL-01_20240119042539_04792201_006_01.h5"
print(in_file)
path_parts = in_file.split("\\")
file_name = path_parts[-1]
del path_parts[-1]

path_parts.append(file_name[0:-3])
out_file = '/'.join(path_parts) + ".csv"
print(out_file)

In [ ]:
with h5py.File(in_file, mode='r') as file:
    print(list(file.keys()))
    beam_enum = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

    df_dict = {
        'Beam': [],
        "Lat": [],
        "Lon": [],
        "FBH": [],
        "FBH Confidence": [],
        "FBH Quality": [],
        "FBH Uncertainty": [],
        "Ground Dist(m)": [],
        "Along Track Dist(m)": [],
        "RS Index (1-based)": [],
        "RS Pos": [],
        "RS Type": [],
        "RS Height(m)": [],
        "RS FB Uncertainty(m)": [],
        "RS Std Dev": [],
        "RS MSS": [],
        "Calendar Time": []
    }

    if "gt1r/freeboard_beam_segment/beam_freeboard" in file:
        filePathToBeamData = "freeboard_beam_segment/beam_freeboard"
    else:
        filePathToBeamData = "freeboard_segment"

    if "gt1r/freeboard_beam_segment/reference_surface_section" in file:
        filePathToReferenceHeights = "freeboard_beam_segment/reference_surface_section"
    else:
        filePathToReferenceHeights = "reference_surface_section"

    print(filePathToBeamData)
    print(filePathToReferenceHeights)

    for beam in beam_enum:
        beamDataPath = f"{beam}/{filePathToBeamData}"
        refDataPath = f"{beam}/{filePathToReferenceHeights}"
        ############################### Store the reference surface data ########################################
        # Read in the distance from the reference surface to the equator
        refSurfPos = file[f'/{refDataPath}/beam_refsurf_dist_x']
        refSurfPos = list(refSurfPos[:])

        # Read in the interpolation flag of the reference surface
        refSurfType = file[f"/{refDataPath}/beam_refsurf_interp_flag"]
        refSurfType = list(refSurfType[:])

        # Read in the uncertainty propogating to the freeboard, based on the reference surface estimate
        refSurfFbUnc = file[f"/{refDataPath}/beam_fb_unc_refsurf"]
        refSurfFbUnc = list(refSurfFbUnc[:])

        # Read in mean sea surface height (the base elevation everything is referring to)
        mss = file[f"/{refDataPath}/beam_refsurf_mss"]
        mss = list(mss[:])

        # Read in the inferred ice elevation (the base elevation for the snow is referring to)
        refSurfHeight = file[f"/{refDataPath}/beam_refsurf_height"]
        refSurfHeight = list(refSurfHeight[:])

        # Read in the standard deviation of the refsurf height
        refSurfStdDev = file[f'/{refDataPath}/beam_refsurf_sigma']
        refSurfStdDev = list(refSurfStdDev[:])
        #######################################################################      

        # Read in the lat/lon
        latitude = file[f'/{beamDataPath}/latitude']
        latitude = list(latitude[:])
        longitude = file[f'/{beamDataPath}/longitude']
        longitude = list(longitude[:])

        # Read in the freeboard height
        fbHeight = file[f'/{beamDataPath}/beam_fb_height']
        fbHeight = list(fbHeight[:])

        # Read in the fbh confidence
        fbConf = file[f'/{beamDataPath}/beam_fb_confidence']
        fbConf = list(fbConf[:])

        # Read in the quality flag
        fbQuality = file[f"/{beamDataPath}/beam_fb_quality_flag"]
        fbQuality = list(fbQuality[:])

        # Read in the total uncertainty from the refSurf + height
        fbUnc = file[f"/{beamDataPath}/beam_fb_unc"]
        fbUnc = list(fbUnc[:])

        # Read in the ref height index
        refSurfIdxs = file[f'/{beamDataPath}/beam_refsurf_ndx']
        refSurfIdxs = list(refSurfIdxs[:])

        # Read in along track distance - used to see how far in meters from refSurface
        alongTrackDist = file[f'/{beamDataPath}/seg_dist_x']
        alongTrackDist = list(alongTrackDist[:])

        refSurfPositions = []
        refSurfTypes = []
        refSurfMss = []
        refSurfHeights = []
        refSurfFbUncertainties = []
        refSurfStDevs = []
        for i in range(len(refSurfIdxs)):
            rsIdx = refSurfIdxs[i]-1
            pointRefSurfPosition  = refSurfPos[rsIdx]
            pointRefSurfType = refSurfType[rsIdx]
            pointMss = mss[rsIdx]
            pointRefSurfHeight = refSurfHeight[rsIdx]
            pointStdDev = refSurfStdDev[rsIdx]
            pointUncertainty = refSurfFbUnc[rsIdx]

            refSurfPositions.append(pointRefSurfPosition)
            refSurfTypes.append(pointRefSurfType)
            refSurfMss.append(pointMss)
            refSurfHeights.append(pointRefSurfHeight)
            refSurfStDevs.append(pointStdDev)
            refSurfFbUncertainties.append(pointUncertainty)

        # print("Max refSurf Idx: ", max(refSurfIdxs))
        # print("Min refSurf Idx: ", min(refSurfIdxs))

        # Calculate ground distance
        distance = []
        x1 = longitude[0]
        y1 = latitude[0]

        for i in range(len(latitude)):
            x2 = longitude[i]
            y2 = latitude[i]
            deltaDistance = hs.haversine((y1,x1), (y2, x2), normalize=True, unit="m")
            distance.append(deltaDistance)
            x1= x2
            y1 = y2

        # Calculate the times
        timevar = file[f'/{beamDataPath}/delta_time']
        time = list(timevar[:])
        dateTime = []
        GPS_EPOCH = datetime.datetime(2018,1,1).timestamp()    # Passing in the ATLAS EPOCH

        for timeInSec in time:
            calendarTime = datetime.datetime.fromtimestamp(timeInSec+GPS_EPOCH).strftime('\"%H:%M:%S.%f %Y-%m-%d\"')
            dateTime.append(calendarTime)

        # We want to append the lat/lon, fbheights, ground distance, refSurfance data for each beam, and then dateTime only on the las

        for i in range(len(latitude)):
            df_dict['Beam'].append(beam)
            df_dict["Lat"].append(latitude[i])
            df_dict["Lon"].append(longitude[i])
            df_dict["FBH"].append(fbHeight[i])
            df_dict["FBH Confidence"].append(fbConf[i])
            df_dict["FBH Quality"].append(fbQuality[i])
            df_dict["FBH Uncertainty"].append(fbUnc[i])
            df_dict["Ground Dist(m)"].append(distance[i])
            df_dict["Along Track Dist(m)"].append(alongTrackDist[i])
            df_dict["RS Index (1-based)"].append(refSurfIdxs[i])
            df_dict["RS Pos"].append(refSurfPositions[i])
            df_dict["RS Type"].append(refSurfTypes[i])
            df_dict["RS Height(m)"].append(refSurfHeights[i])
            df_dict["RS FB Uncertainty(m)"].append(refSurfFbUncertainties[i])
            df_dict["RS Std Dev"].append(refSurfStDevs[i])
            df_dict["RS MSS"].append(refSurfMss[i])
            df_dict["Calendar Time"].append(dateTime[i])

        for col, data in df_dict.items():
            print(beam)
            print(f"{col}:{len(data)}")




In [ ]:
                
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in df_dict.items()]))
df = df.replace(r'^\s*$', np.nan, regex=True)

print(df.head(5))
print(df.shape)
# Drop all rows where ICESAT has no FB height data
val = df.iloc[0]['FBH']
filtered = df[ df['FBH'] != val ]
print(filtered.head(5))
print(filtered.shape)


filtered.to_csv(out_file, date_format=None)

print("Wrote to CSV")
print("\n")